# Online PCA

Los métodos para la reducción de dimensiones, en especial los métodos del tipo __online__, siguen siendo un campo de investigación.

En específico, PCA tiene varios métodos que intentan competir por convertirse en estandard. En este notebook vamos a ver uno de los métodos más populares llamado __incremental pca__.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#--para cargar los datos
from sklearn.datasets import load_iris

#--funciones relacionadas a PCA
from sklearn.decomposition import PCA, IncrementalPCA

In [ ]:
#--cargamos los datos
iris = load_iris()

#--atributos: los datos a los que reduciremos las dimensiones
X = iris.data
#--variable respuesta: la utilizó solo para fines didacticos al
# mostrar la respuesta
y = iris.target

X[:5, ]

In [ ]:
#--definimos el número de dimensiones objetivo
n_components = 2

Calculamos PCA "normal" para comparar el resultado con el algoritmo PCA en línea.

In [ ]:
#--definimos una instancia de PCA
pca = PCA(n_components=n_components)

#--calculamos la reducción
X_pca = pca.fit_transform(X)

Primero, vamos a utilizar __incremental PCA__ en todo el data set cargado en memoria.

¿Porqué desearíamos hacer lo anteior si ya existe la función __PCA__? Dependiendo del tamaño de __batch_size__, incremental PCA puede ser más eficiente que PCA en terminos de uso de memoria, además de que permite el uso de _sparse arrays_.

Por default el __batch_size__ se define como 5 veces el número de atributos. El cuál es un buen balance en terminos de precisión de la respuesta y uso de memoria.

In [ ]:
#--entrenamos y transformamos los datos con incremental PCA
ipca = IncrementalPCA(n_components=n_components, batch_size=10)

X_ipca = ipca.fit_transform(X)

Finalmente utilizamos __partial_fit__ cuando entrenemos el algoritmo en línea.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#--leemos el archivo desde un csv para leerlo por partes
file_iris = '/content/drive/MyDrive/data_sets/iris.csv'

#--definimos una instancia de ipca
ipca2 = IncrementalPCA(n_components=n_components)

#--realizamos el entrenamiento
reader = pd.read_csv(file_iris, iterator = True)
n = 0 #--contar el número de filas
try:
    #--iteramos hasta el final
    while reader:
        X_batch = reader.get_chunk(10)
        ipca2.partial_fit(X_batch.values[:,:4])
        n += X_batch.shape[0]
except StopIteration:
    pass
finally:
    del reader

#--transformamos los datos
reader = pd.read_csv(file_iris, iterator = True)
i = 0 #--indice auxiliar para rellenar el resultado
X_reduced = np.zeros((n,2))
try:
    #--iteramos hasta el final
    while reader:
        X_batch = reader.get_chunk(10)
        X_reduced[i:(i+X_batch.shape[0]), :] = ipca2.transform(X_batch.values[:,:4])
        i += X_batch.shape[0]
except StopIteration:
    pass
finally:
    del reader

Comparamos los resultados por medio de gráficas las dimensiones reducidas.

In [ ]:
#--definimos un color para cada clase. Recordemos que iris dataset tiene tres clases
colors = ['navy', 'turquoise', 'darkorange']

j = 1
plt.figure(figsize=(15,6))
for X_transformed, title in [(X_pca, "PCA"), (X_ipca, "Incremental PCA"), (X_reduced, "Incremental PCA online")]:
    plt.subplot(1,3,j)
    for color, i, target_name in zip(colors, [0, 1, 2], iris.target_names):
        plt.scatter(X_transformed[y == i, 0], X_transformed[y == i, 1],
                    color=color, lw=2, label=target_name)

    j = j+1
    if "online" in title:
        err = np.abs(np.abs(X_pca) - np.abs(X_reduced)).mean()
        plt.title(title + " del iris dataset\n Error absoluto promedio "
                  "%.4f" % err)
    elif "Incremental" in title:
        err = np.abs(np.abs(X_pca) - np.abs(X_ipca)).mean()
        plt.title(title + " del iris dataset\n Error absoluto promedio "
                  "%.4f" % err)
    else:
        plt.title(title + " del iris dataset")

    plt.legend(loc="best")
    plt.axis([-4, 4, -1.5, 1.5])

plt.show()